In [1]:
from geoband.API import *
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import *
import datetime
#pd.set_option('display.max_rows', None)

/opt/app-root/lib/python3.6/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [2]:
police = gpd.read_file('1.5개시도_경찰서_관할경계.geojson')
cctv = pd.read_csv('10.5개시도_CCTV설치현황.csv')
cctv_raw = cctv

In [3]:
df = pd.read_csv('base_df.csv')
plist = df['jur_stn'].unique()
yearseries = df['date'].unique()
yearseries = yearseries.tolist()
yearseries.append('2016(이전)')

In [4]:
plist

array(['서울중부경찰서', '서울종로경찰서', '서울남대문경찰서', '서울서대문경찰서', '서울혜화경찰서', '서울용산경찰서',
       '서울성북경찰서', '서울동대문경찰서', '서울마포경찰서', '서울영등포경찰서', '서울성동경찰서', '서울동작경찰서',
       '서울광진경찰서', '서울서부경찰서', '서울강북경찰서', '서울금천경찰서', '서울중랑경찰서', '서울강남경찰서',
       '서울관악경찰서', '서울강서경찰서', '서울강동경찰서', '서울종암경찰서', '서울구로경찰서', '서울서초경찰서',
       '서울양천경찰서', '서울송파경찰서', '서울노원경찰서', '서울방배경찰서', '서울은평경찰서', '서울도봉경찰서',
       '서울수서경찰서', '수원중부경찰서', '수원남부경찰서', '수원서부경찰서', '세종경찰서', '창원중부경찰서',
       '창원서부경찰서', '마산중부경찰서', '마산동부경찰서', '진주경찰서', '진해경찰서'], dtype=object)

In [5]:
yearseries

['2017(상반기)',
 '2017(하반기)',
 '2018(상반기)',
 '2018(하반기)',
 '2019(상반기)',
 '2019(하반기)',
 '2020(상반기)',
 '2020(하반기)',
 '2016(이전)']

In [6]:
cctv['lon'] = cctv['lon'].astype(float)
cctv['lat'] = cctv['lat'].astype(float)
cctv['geometry'] = cctv.apply(lambda row:Point([row['lon'],row['lat']]), axis=1)
cctv = gpd.GeoDataFrame(cctv,geometry='geometry',crs="EPSG:4326")

cctv = gpd.sjoin(cctv,police,how='left',op='within')
cctv.drop(['index_right','PNAME'],axis=1,inplace=True)
          
cctv.rename(columns = {"NAME":"jur_stn"}, inplace=True)

In [9]:
cctv['set_yr'].value_counts()

`2020-01    1564
`2015-12    1272
`2019-12    1264
`2018-12    1180
`2013-12    1112
            ... 
`2006-07       1
`2006-06       1
`2009-05       1
`2008-06       1
`2008-05       1
Name: set_yr, Length: 188, dtype: int64

In [10]:
cctv.dropna(subset=['jur_stn'],inplace=True)

cctv_set_yr = cctv['set_yr'].fillna("`1900-01")
cctv.drop(['set_yr'],axis=1,inplace=True)
cctv['set_yr'] = cctv_set_yr

cctv

,address,purpose,cctv_cnt,lat,lon,geometry,jur_stn,set_yr
0,세종특별자치시 조치원읍 교리 129-1,다목적,3,36.605059,127.296286,POINT (127.29629 36.60506),세종경찰서,`1900-01
1,세종특별자치시 조치원읍 남리144,다목적,2,36.597678,127.302452,POINT (127.30245 36.59768),세종경찰서,`1900-01
2,세종특별자치시 조치원읍 남리 425,다목적,3,36.594863,127.300492,POINT (127.30049 36.59486),세종경찰서,`1900-01
3,세종특별자치시 조치원읍 명리82,다목적,3,36.600620,127.301301,POINT (127.30130 36.60062),세종경찰서,`1900-01
4,세종특별자치시 조치원읍 상리18-10,다목적,4,36.602434,127.303145,POINT (127.30315 36.60243),세종경찰서,`1900-01
...,...,...,...,...,...,...,...,...
46198,서울특별시 성북구 길음동 635-7,어린이보호,4,37.613782,127.018663,POINT (127.01866 37.61378),서울강북경찰서,`2010-12
46199,서울특별시 성북구 길음동 1280-4,어린이보호,4,37.610174,127.015332,POINT (127.01533 37.61017),서울성북경찰서,`2010-12
46200,서울특별시 성북구 길음동 1168,어린이보호,4,37.609928,127.022334,POINT (127.02233 37.60993),서울성북경찰서,`2010-12
46201,서울특별시 성북구 하월곡동 87-6,어린이보호,4,37.609267,127.032080,POINT (127.03208 37.60927),서울종암경찰서,`2010-12


In [11]:
reset_yr = []

for i in cctv['set_yr'].tolist():
    date = datetime.datetime.strptime(i,"`%Y-%m").date()
    reset_yr.append(date)
reset_yr

[datetime.date(1900, 1, 1),
 datetime.date(1900, 1, 1),
 datetime.date(1900, 1, 1),
 datetime.date(1900, 1, 1),
 datetime.date(1900, 1, 1),
 datetime.date(1900, 1, 1),
 datetime.date(1900, 1, 1),
 datetime.date(1900, 1, 1),
 datetime.date(1900, 1, 1),
 datetime.date(1900, 1, 1),
 datetime.date(1900, 1, 1),
 datetime.date(1900, 1, 1),
 datetime.date(1900, 1, 1),
 datetime.date(1900, 1, 1),
 datetime.date(1900, 1, 1),
 datetime.date(1900, 1, 1),
 datetime.date(1900, 1, 1),
 datetime.date(1900, 1, 1),
 datetime.date(1900, 1, 1),
 datetime.date(1900, 1, 1),
 datetime.date(1900, 1, 1),
 datetime.date(1900, 1, 1),
 datetime.date(1900, 1, 1),
 datetime.date(1900, 1, 1),
 datetime.date(1900, 1, 1),
 datetime.date(1900, 1, 1),
 datetime.date(1900, 1, 1),
 datetime.date(1900, 1, 1),
 datetime.date(1900, 1, 1),
 datetime.date(1900, 1, 1),
 datetime.date(1900, 1, 1),
 datetime.date(1900, 1, 1),
 datetime.date(1900, 1, 1),
 datetime.date(1900, 1, 1),
 datetime.date(1900, 1, 1),
 datetime.date(1900,

In [12]:
cctv.drop(['set_yr'],axis=1,inplace=True)    
cctv['set_yr'] = reset_yr

In [13]:
year = []

for idx, row in cctv.iterrows():
    if row['set_yr'] <= datetime.date(2016,12,31):
        yr = '2016(이전)'
    elif row['set_yr'] <= datetime.date(2017,6,30):
        yr = '2017(상반기)'
    elif row['set_yr'] <= datetime.date(2017,12,31):
        yr = '2017(하반기)'
    elif row['set_yr'] <= datetime.date(2018,6,30):
        yr = '2018(상반기)'
    elif row['set_yr'] <= datetime.date(2018,12,31):
        yr = '2018(하반기)'
    elif row['set_yr'] <= datetime.date(2019,6,30):
        yr = '2019(상반기)'
    elif row['set_yr'] <= datetime.date(2019,12,31):
        yr = '2019(하반기)'
    else:
        yr = 'unknown'
    #print(ym)
    year.append(yr)

In [14]:
cctv['year'] = year
cctv.drop(['set_yr','address','purpose','lat','lon','geometry'],axis=1,inplace=True)
cctv = cctv[cctv['year'] != 'unknown']
cctv = pd.DataFrame(cctv)

cctvjur = cctv.groupby(['jur_stn','year'])[['cctv_cnt']].sum()

for i in plist:
    for j in yearseries:
        if not cctvjur.index.isin([(i, j)]).any():
            cctvjur.loc[(i, j), "cctv_cnt"] = int(0)
            
cctvjur=cctvjur.reset_index('year')
cctvjur=cctvjur.reset_index('jur_stn')
cctvjur.sort_values(['jur_stn', 'year'], ascending=[True, True],inplace=True)
cctvjur_final = pd.DataFrame(
                  index=[cctvjur['jur_stn'], cctvjur['year']],
                  columns=['cctv'])
cctvjur_final['cctv'] = cctvjur['cctv_cnt'].tolist()
cctvjur_final['cctv'] = cctvjur_final['cctv'].astype(int)
cctv=cctvjur_final.reset_index('year')
cctv=cctv.reset_index('jur_stn')

In [15]:
pd.set_option('display.max_rows', None)
cctv

,jur_stn,year,cctv
0,마산동부경찰서,2016(이전),504
1,마산동부경찰서,2017(상반기),79
2,마산동부경찰서,2017(하반기),107
3,마산동부경찰서,2018(상반기),170
4,마산동부경찰서,2018(하반기),58
5,마산동부경찰서,2019(상반기),154
6,마산동부경찰서,2019(하반기),84
7,마산동부경찰서,2020(상반기),0
8,마산동부경찰서,2020(하반기),0
9,마산중부경찰서,2016(이전),340


In [18]:
cctv.to_csv("10.cctv.csv", index=False)